## Enzyme Kinetics from csv values

This notebook demonstrates how to read enzyme kinetics data from a CSV file and plot a *Michaelis-Menten* curve using matplotlib.

Workflow:
1. Read the CSV file containing enzyme kinetics data.
2. Extract the substrate concentration and reaction rate values.
3. Calculate the initial velocities (V0)
4. Fit the Michaelis-Menten equation to the data.
5. Plot the data and the fitted curve.
6. Annotate the plot with the fitted parameters.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [ ]:

# Michaelis-Menten equation
def michaelis_menten(S, V_max, K_m):
    return (V_max * S) / (K_m + S)

# Load external data from CSV
def load_external_data(csv_file):
    df = pd.read_csv(csv_file)
    return df

In [ ]:
# Constants
epsilon = 1000  # Molar absorptivity (M^-1 cm^-1)
path_length = 1  # Path length (cm)
# Load and process external data
csv_file = './data/simulated_absorbance_data.csv'  # Replace with your CSV file path
df_external = load_external_data(csv_file)

df_external


In [ ]:
# Calculate initial velocities from external data
def calculate_initial_velocities(df, epsilon, path_length):
    substrate_concentrations = df['Substrate_Concentration'].unique()
    initial_velocities = []
    std_velocities = []  # Standard deviation for error bars (if replicates exist)

    for S in substrate_concentrations:
        subset = df[df['Substrate_Concentration'] == S]
        # Convert absorbance to concentration
        subset['Concentration'] = subset['Absorbance'] / (epsilon * path_length)
        # Calculate initial velocity (slope of concentration vs time)
        v0_values = []
        for replicate in subset['Replicate'].unique():  # If replicates exist
            replicate_data = subset[subset['Replicate'] == replicate]
            slope, _ = np.polyfit(replicate_data['Time'], replicate_data['Concentration'], 1)
            v0_values.append(slope)
        # Store mean and standard deviation of initial velocities
        initial_velocities.append(np.mean(v0_values))
        std_velocities.append(np.std(v0_values))

    return substrate_concentrations, initial_velocities, std_velocities

In [ ]:
substrate_concentrations_ext, initial_velocities_ext, std_velocities_ext = calculate_initial_velocities(df_external, epsilon, path_length)

In [ ]:
substrate_concentrations_ext, initial_velocities_ext, std_velocities_ext

In [ ]:
# Simulated data (from previous code)
substrate_concentrations_sim = np.array([10, 20, 50, 100, 200])
initial_velocities_sim = [20, 40, 80, 120, 160]  # Example simulated velocities
std_velocities_sim = [2, 3, 4, 5, 6]  # Example standard deviations

In [ ]:

# Fit Michaelis-Menten equation to external data
popt_ext, _ = curve_fit(michaelis_menten, substrate_concentrations_ext, initial_velocities_ext, p0=[100, 50])

# Generate fitted curve for external data
S_fit = np.linspace(0, max(substrate_concentrations_ext), 500)
v_fit_ext = michaelis_menten(S_fit, *popt_ext)

# Plot the Michaelis-Menten fit with external data
plt.figure(figsize=(12, 8))

# Simulated data (with error bars)
plt.errorbar(substrate_concentrations_sim, initial_velocities_sim, yerr=std_velocities_sim, 
             fmt='o', color='blue', label='Simulated Data (Mean ± SD)', capsize=5)

# External data (with error bars)
plt.errorbar(substrate_concentrations_ext, initial_velocities_ext, yerr=std_velocities_ext, 
             fmt='s', color='red', label='External Data (Mean ± SD)', capsize=5)

# Fitted curve for external data
plt.plot(S_fit, v_fit_ext, 'r--', label=f'External Fit: V_max={popt_ext[0]:.2f}, K_m={popt_ext[1]:.2f}', linewidth=2)

plt.xlabel('Substrate Concentration [S] (µM)')
plt.ylabel('Initial Velocity v0 (µM/s)')
plt.title('Michaelis-Menten Fit with External Data')
plt.legend()
plt.show()

In [ ]:
# save the plot
plt.savefig('michaelis_menten_fit_external_data.svg', dpi=300, bbox_inches='tight')